#### pgmpy을 사용한 베이즈 정리 적용¶
pgmpy 패키지에서는 조건부 확률을 정의할 수 있는 TabularCPD 클래스와 베이즈 정리를 적용할 수 있는 BayesianModel 클래스를 제공한다.

#### 검사 시약 문제
- 사건
```
병에 걸리는 경우: 사건  D 
양성 반응 보이는 경우: 사건  S 
병에 걸린 사람이 양성 반응을 보이는 경우: 조건부 사건 S|D 
양성 반응 보이는 사람이 병에 걸려 있을 경우: 조건부 사건 D|S 
```
- 문제
P(S|D)=0.99 가 주어졌을 때,  P(D|S) 를 구하라

TabularCPD 클래스는 다음처럼 사용한다.

- TabularCPD(variable, variable_card, value, evidence, evidence_card)
- variable: 확률 변수의 이름 문자열
- variable_card: 확률 변수가 가질 수 있는 경우의 수
- value: 조건부 확률 배열. 하나의 열이 하나의 조건(경우)을 뜻하므로 하나의 열의 확률 합은 1이어야 한다.
- evidence: 조건이 되는 확률 변수의 이름 문자열의 리스트
- evidence_card: 조건이 되는 확률 변수가 가질 수 있는 경우의 수의 리스트

In [3]:
from pgmpy.factors.discrete import TabularCPD

우선 확률 변수 X를 이용하여 병에 걸렸을 확률  P(D)=P(X=1) , 병에 걸리지 않았을 확률  P(DC)=P(X=0) 를 정의한다.

In [4]:
cpd_X = TabularCPD('X', 2, [[1 - 0.002, 0.002]])
print(cpd_X)

╒═════╤═══════╕
│ X_0 │ 0.998 │
├─────┼───────┤
│ X_1 │ 0.002 │
╘═════╧═══════╛


다음으로 확률 변수 Y를 이용하여 양성 반응이 나올 확률  P(S)=P(Y=1) , 음성 반응이 나올 확률  P(SC)=P(Y=0) 를 정의한다. 이 확률은 병에 걸렸는가 걸리지 않았는가에 따라 달라지므로 다음처럼 조건부로 정의한다.

In [5]:
cpd_YX = TabularCPD('Y', 2, np.array([[0.95, 0.01], [0.05, 0.99]]),
                    evidence=['X'], evidence_card=[2])
print(cpd_YX)

╒═════╤══════╤══════╕
│ X   │ X_0  │ X_1  │
├─────┼──────┼──────┤
│ Y_0 │ 0.95 │ 0.01 │
├─────┼──────┼──────┤
│ Y_1 │ 0.05 │ 0.99 │
╘═════╧══════╧══════╛


이 조건부 확률과 조건이 되는 확률을 결합하기 위해서는 BayesianModel 클래스 객체를 만들고 add_cpds 메서드로 위에서 구현한 조건부 확률을 추가한다. check_model 메서드로 모형이 완전한지 확인할 수 있다.

In [6]:
from pgmpy.models import BayesianModel

model = BayesianModel([('X', 'Y')])
model.add_cpds(cpd_X, cpd_YX)
model.check_model()

True

BayesianModel 클래스는 변수 제거법(VariableElimination)을 사용한 추정을 제공한다. 사용법은 다음과 같다. 자세한 내용은 추후 확률적 그래프 모형에서 다루게 된다.

In [7]:
from pgmpy.inference import VariableElimination

infer = VariableElimination(model)
posterior = infer.query(['X'], evidence={'Y': 1})
print(posterior['X'])

╒═════╤══════════╕
│ X   │   phi(X) │
╞═════╪══════════╡
│ X_0 │   0.9618 │
├─────┼──────────┤
│ X_1 │   0.0382 │
╘═════╧══════════╛


#### 과일 선택 문제¶
어떤 과일 가게에서 사과와 오렌지를 팔고 있다. 사과와 오렌지는 Red 농장과 Blue 농장에서 공급받는다.

어느날 Red 농장에서 붉은 상자에 사과 2개, 오렌지 6개를 담아서 보냈다. Blue 농장에서는 푸른 상자에 사과가 3, 오렌지가 1개를 담아서 공급했다.

이 때 어떤 손님이 와서 오렌지를 하나 선택했다( X=  오렌지). 이 오렌지는 Red 농장에서 온 것일까( Y= Red), 아니면 Blue 농장에서 온 것일까( Y= Blue)? 만약 Red 농장과 Blue 농장 중 하나의 답을 정해야 한다면 여러분은 어느 농장을 선택하겠는가?
```
손님이 선택한 과일이 사과인 사건  X=A 
손님이 선택한 과일이 오렌지인 사건  X=O 
손님이 선택한 과일이 Red 농장에서 왔을 사건  Y=R 
손님이 선택한 과일이 Blue 농장에서 왔을 사건  Y=B
```

여기에서는 확률 변수 Y를 Y=0인 경우가 Red 농장, Y=1인 경우가 Blue 농장으로 정의한다.

In [8]:
cpd_Y = TabularCPD('Y', 2, [[8/12, 4/12]])
print(cpd_Y)

╒═════╤══════════╕
│ Y_0 │ 0.666667 │
├─────┼──────────┤
│ Y_1 │ 0.333333 │
╘═════╧══════════╛


확률 변수 X는 X=0인 경우를 오렌지인 경우, X=1인 경우를 사과인 경우으로 정의한다.

In [9]:
cpd_XY = TabularCPD('X', 2, np.array([[6/8, 1/4], [2/8, 3/4]]),
                    evidence=['Y'], evidence_card=[2])
print(cpd_XY)

╒═════╤══════╤══════╕
│ Y   │ Y_0  │ Y_1  │
├─────┼──────┼──────┤
│ X_0 │ 0.75 │ 0.25 │
├─────┼──────┼──────┤
│ X_1 │ 0.25 │ 0.75 │
╘═════╧══════╧══════╛


베이지안 네트워크 모형으로 사후확률을 계산하면  0.1429=1/7 이 나온다.

In [11]:
from pgmpy.models import BayesianModel

model = BayesianModel([('Y', 'X')])
model.add_cpds(cpd_Y, cpd_XY)
model.check_model()

True

In [12]:
from pgmpy.inference import VariableElimination

infer = VariableElimination(model)
posterior = infer.query(['Y'], evidence={'X': 0})
print(posterior['Y'])

╒═════╤══════════╕
│ Y   │   phi(Y) │
╞═════╪══════════╡
│ Y_0 │   0.8571 │
├─────┼──────────┤
│ Y_1 │   0.1429 │
╘═════╧══════════╛


In [13]:
1/7

0.14285714285714285